In [1]:
# import the necessary libraries
import pandas as pd
import sqlite3


# Extract data from the users database


In [2]:
def extract(db_path='users.db'):
    conn = sqlite3.connect(db_path)
 
    # Read expenses data 
    expenses_df = pd.read_sql_query("""
        SELECT user_id, category, SUM(amount) as total_spent
        FROM expenses
        GROUP BY user_id, category
    """, conn)

    # Read budgets data
    budgets_df = pd.read_sql_query("""
        SELECT user_id, category, amount as budgeted_amount
        FROM budgets
    """, conn)
    conn.close()
    return expenses_df, budgets_df

# Transform the data to get it ready for being loaded into the final dataset


In [3]:
def transform(expenses_df,budgets_df):
     # Merge budgets with expenses on user_id and category
    merged_df = pd.merge(
        budgets_df,
        expenses_df,
        on=['user_id', 'category'],
        how='left'
    )

    # Fill NaN values in total_spent with 0 for categories with no spending
    merged_df['total_spent'] = merged_df['total_spent'].fillna(0)

    # Calculate difference between spending and budget
    merged_df['difference'] = merged_df['total_spent'] - merged_df['budgeted_amount']

    # Add status column
    merged_df['status'] = merged_df['difference'].apply(
        lambda x: 'Over Budget' if x > 0 else 'Within Budget'
    )

    return merged_df

# Load the data into a transformed dataset

In [4]:
def load():
    expenses_df, budgets_df = extract()
    result_df = transform(expenses_df, budgets_df)

    # Save to CSV or database, or print
    result_df.to_csv('etl/output/budget_vs_expenses.csv', index=False)
    print("Budget analysis saved.")

In [5]:
if __name__ == "__main__":
    load()

DatabaseError: Execution failed on sql '
        SELECT user_id, category, SUM(amount) as total_spent
        FROM expenses
        GROUP BY user_id, category
    ': no such table: expenses